In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
jdk_version = '11'
hadoop_version = 'hadoop2.7'

# Set Environment Variables
os.environ['SPARK_VERSION'] = spark_version
os.environ['JDK_VERSION'] = jdk_version
os.environ['HADOOP_VERSION'] = hadoop_version
os.environ['JAVA_HOME'] = f'/usr/lib/jvm/java-{jdk_version}-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'

# Update modules and install Spark and Java
!apt-get update
!apt-get install openjdk-$JDK_VERSION-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [809 kB]
Hit:13 http://ppa.laun

In [3]:
postgresql_java_version = '42.2.9'
os.environ['POSTGRESQL_JAVA_VERSION'] = postgresql_java_version
!wget https://jdbc.postgresql.org/download/postgresql-$POSTGRESQL_JAVA_VERSION.jar

--2021-11-23 00:48:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2021-11-23 00:48:42 (1.31 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level-One-Starter-Code-2").config(
    "spark.driver.extraClassPath",
    f"/content/postgresql-42.2.9.jar"
  ).getOrCreate()

# Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
# Read and show the data
software_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
software_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# What is the size of Data?
software_df.select("customer_id").count()

341931

# Cleaned up DataFrames to match tables

In [37]:
from pyspark.sql.functions import *
# Review DataFrame
software_df = software_df.withColumn("review_date",to_date(col("review_date"),"yyyy-MM-dd"))
software_df.cache()
software_df = software_df.withColumn("star_rating",col("star_rating").cast("int"))
software_df.cache()
software_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [21]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

schema = [StructField("marketplace", StringType(), True),
          StructField("customer_id", IntegerType(), True),
          StructField("review_id", StringType(), True),
          StructField("product_id", StringType(), True),
          StructField("product_parent", IntegerType(), True),
          StructField("product_title", StringType(), True),
          StructField("product_category", StringType(), True),
          StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True),
          StructField("total_votes", IntegerType(), True),
          StructField("vine", StringType(), True),
          StructField("verified_purchase", StringType(), True),
          StructField("review_headline", StringType(), True),
          StructField("review_body", StringType(), True),
          StructField("review_date", DateType(), True),]

final=StructType(fields=schema)

In [22]:
review_id_table_df = software_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id_table_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|42605767   |B00MUTIDKI|248732228     |2015-08-31 |
|R12NR0R5A9F7FT|51771800   |B00EPACNUG|531462352     |2015-08-31 |
|R1LSH74R9XAP59|16053526   |B00164AZA4|473982505     |2015-08-31 |
|R1QXUNTF76K7L6|15319481   |B00E6LIEFM|189774198     |2015-08-31 |
|R2F7DR75PS8NKT|1441820    |B00VWEBG06|852470365     |2015-08-31 |
|R2C1DJSCC8UFS6|37107850   |B00EP7AP7C|279360628     |2015-08-31 |
|R1AXGS1W4YFXMX|302120     |B00OW2PET4|729971168     |2015-08-31 |
|R1XU1B93402SYJ|20193077   |B00N4OLCRO|776572654     |2015-08-31 |
|R2U432NB3OPVR0|13106017   |B005CELN8W|222071424     |2015-08-31 |
|R3R6FIMIOQ5SP9|32587108   |B005CELL1G|168801430     |2015-08-31 |
|R12TX6V09C9QNQ|32020017   |B00H09BOXQ|455502995     |2015-08-31 |
|R33UCII6YKUMKV|13376158   |B00MCLGAAO|652069521     |2015-08-

In [23]:
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [24]:
from pyspark.sql import Row
products_table_df = software_df.dropDuplicates((["product_id"])).select("product_id", "product_title")

products_table_df.show(truncate=False)

+----------+--------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                     |
+----------+--------------------------------------------------------------------------------------------------+
|031024837X|The Expositor's Bible Commentary 5.0 for Windows : The Complete Award-Winning 12-Volume Commentary|
|0838447384|Triple Play Plus Spanish                                                                          |
|097680428X|College Prep Genius DVD: The No Brainer Way to SAT* Success - Updated and Revised                 |
|1618945157|Trainz: A New Era                                                                                 |
|1881901912|Learn Sicilian / Mparamu lu sicilianu DVD version                                                 |
|9650706666|Encyclopedia Judaica CD-ROM Edition                                                         

In [25]:
products_table_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [26]:
customers_table_df = software_df.select("customer_id").groupBy("customer_id").count().withColumnRenamed("count","customer_count")
customers_table_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15634680|             1|
|   44696507|             1|
|   19901367|             1|
|   11337682|             1|
|   52021773|             1|
|   29916198|             1|
|   42030944|             1|
|   52765209|             1|
|   13519509|             2|
|   39320350|             5|
|   23249582|             1|
|   46351928|             3|
|   44179620|             1|
|     637252|             1|
|   34038793|             1|
|   51136490|             1|
|   49354815|             1|
|     149761|             2|
|   43333798|             1|
|    4657027|             1|
+-----------+--------------+
only showing top 20 rows



In [27]:
customers_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [28]:
vine_table_df = software_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|1          |2            |2          |N   |
|R12NR0R5A9F7FT|5          |0            |0          |N   |
|R1LSH74R9XAP59|2          |0            |1          |N   |
|R1QXUNTF76K7L6|2          |0            |0          |N   |
|R2F7DR75PS8NKT|5          |0            |0          |N   |
|R2C1DJSCC8UFS6|3          |0            |0          |N   |
|R1AXGS1W4YFXMX|1          |0            |2          |N   |
|R1XU1B93402SYJ|1          |1            |1          |N   |
|R2U432NB3OPVR0|5          |0            |0          |N   |
|R3R6FIMIOQ5SP9|5          |0            |0          |N   |
|R12TX6V09C9QNQ|5          |0            |0          |N   |
|R33UCII6YKUMKV|3          |2            |2          |N   |
|RZKDAB9TGO053 |1          |0            |0          |N   |
|R2EMN2EEDN73ZA|4          |0           

In [29]:
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Push to AWS RDS instance

In [33]:
mode = "append"
jdbc_url = "jdbc:postgresql://big-data-1.cluyaga9jyra.us-east-2.rds.amazonaws.com:5432/big_data_1"
config = {
    "user" : "{username}", 
    "password" : "{password}",
    "driver" : "org.postgresql.Driver"
}

In [34]:
# Write review_id_df to table in RDS
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [39]:
# Write products_df to table in RDS
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [40]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)